In [73]:
# imports 

import pandas as pd
import numpy as np 
import re 
from functools import reduce
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
from datetime import datetime

# general plot settings 
sns.set(rc={'figure.figsize':(16,5)})
sns.set_style('ticks')

In [185]:
# used functions 

def get_Consumer_description_topics(data_raw):
    description  = data_raw.columns[0]
    
    # columns of feature description 
    b = re.findall("[^\n]+:", description)
    b1 = b[:4]+ b[5:]
    b2 = [re.findall("[A-Za-z ]+", b1[i]) for i in range(len(b1))]
    b3 = b2[:2] + b2[3:]
    b4 = [el for [el] in b3]
    b4.append(b2[2][0])
    b4.append(b2[2][3])
    features = [b4[i].strip( ' ' ) for i in range(len(b4))]
    
    return features


def get_Consumer_description_content(data_raw):
    description  = data_raw.columns[0]
    
    # contect of feature description 
    d = re.findall(":[^\n]+", description)
    e = d[:2] + d[3:]
    period = re.findall("[0-9]+-[A-Za-z]+-[0-9]+", d[2])
    f = [re.findall("[^:]+", e[i]) for i in range(len(e))]
    g = [el for [el] in f]
    g.append(period[0])
    g.append(period[1])
    features_description = [g[i].strip( ' ' ) for i in range(len(g))]
    
    return features_description



def get_Consumer_description_content_string(data_raw):
    #description  = data_raw.columns[0]
    
    # contect of feature description 
    d = re.findall(":[^\n]+", data_raw)
    e = d[:2] + d[3:]
    period = re.findall("[0-9]+-[A-Za-z]+-[0-9]+", d[2])
    f = [re.findall("[^:]+", e[i]) for i in range(len(e))]
    g = [el for [el] in f]
    g.append(period[0])
    g.append(period[1])
    features_description = [g[i].strip( ' ' ) for i in range(len(g))]
    
    return features_description



def make_dict_appliances (list_of_Major_Charges):
    a = list_of_Major_Charges.split(',')
    b = [el.strip( ' ' ) for el in a]
    c = [el.replace('.', '') for el in b]
    d = []
    e = []
    [d.append(c[i][0]) for i in range(len(c))]
    [e.append(c[i][1:]) for i in range(len(c))]
    f = [el.strip( ' ' ) for el in e]
    dictionary = dict(zip(f, d))
    #pd.DataFrame.from_dict(dictionary, orient='index').T
    return dictionary

def make_dict_column(data):
    x = list(data['Major Charges'])
    y = []
    [y.append(make_dict_appliances(el)) for el in x]
    return y

# make form long list of variable decsribed apliences a short list 
def clean_keys (x): 
    for i in range(len(x)):
        a = re.findall('TV|LCD', x[i])
        b = re.findall('desktop computer|monitor|laptop', x[i])
        c = re.findall('cleaner', x[i])
        d = re.findall('kettle', x[i])
        e = re.findall('wash', x[i])
        f = re.findall('heater', x[i])
        g = re.findall('fridge', x[i])
        h = re.findall('laptop', x[i]) 
        #print(z)
        if a != []: 
            x[i] = 'tv'
        elif b != []: 
            x[i] = 'laptop'
        elif c != []: 
            x[i] = 'cleaner'
        elif d != []: 
            x[i] = 'kettle'
        elif e != []: 
            x[i] = 'washing maschine'
        elif f != []: 
            x[i] = 'heater'
        elif g != []: 
            x[i] = 'fridge'
            
    return x 

# rename rare apliences to others 
def make_other (x, other_list): 
    for i in range(len(list(x))):
        if x[i] in other_list:
            x[i] = other_list
    return x 


In [75]:
# loading descriptions from data sets 

house_1 = pd.read_excel("data/Private Home_1_2011_June_03_to_17_2.xlsx", sheet_name='Info')
house_2 = pd.read_excel("data/Private Home_2_2012_July_16_to_26.xls", sheet_name='Folha1')
house_3 = pd.read_excel("data/Private Home_3_2013_January_23_to_February_07.xls", sheet_name='Info')
house_4 = pd.read_excel("data/Private Home_4_2014_January_06_to_30.xls", sheet_name='Info')
house_5 = pd.read_excel("data/Private Home_5_2011_August_04_to_18.xlsx", sheet_name='Info')
house_6 = pd.read_excel("data/Private Home_6_2011_December_25_to_15.xlsx", sheet_name='Info')
house_7 = pd.read_excel("data/Private Home_7_2011_September_26_to_October_03.xlsx", sheet_name='Info')
house_8 = pd.read_excel("data/Private Home_8_2012_June_01_to_15.xlsx", sheet_name='Info')
house_9 = pd.read_excel("data/Private Home_9_2012_December_29_to_2013_January_12.xlsx", sheet_name='Info')
house_10 = pd.read_excel("data/Private Home_10_Data House10_1minutes_01-06-2012_30-06-2012.xlsx", sheet_name='Info')
house_10 = list(house_10["Unnamed: 0"][2:3])

In [201]:
# merging descriptions dataframe 
Feature_df = pd.DataFrame(get_Consumer_description_content(house_1), 
                          index =get_Consumer_description_topics(house_2), 
                          columns =['House 1']) 

Feature_df['House 2'] = get_Consumer_description_content(house_2)
Feature_df['House 3'] = get_Consumer_description_content(house_3)
Feature_df['House 4'] = get_Consumer_description_content(house_4)
Feature_df['House 5'] = get_Consumer_description_content(house_5)
Feature_df['House 7'] = get_Consumer_description_content(house_7)
Feature_df['House 8'] = get_Consumer_description_content(house_8)
Feature_df['House 9'] = get_Consumer_description_content(house_9)

# the appliance list is missing at house 6 and 10 
NaN = ['NaN']
Feature_df['House 6'] = get_Consumer_description_content(
    house_6)[:5] + NaN + get_Consumer_description_content(house_6)[5:]
Feature_df['House 10'] = get_Consumer_description_content_string(
    house_10[0])[:2] + NaN + get_Consumer_description_content_string(
    house_10[0])[2:4] + NaN + get_Consumer_description_content_string(house_10[0])[4:]

Feature_df = Feature_df.T.reset_index()
Feature_df.rename({'index': 'House'}, axis=1, inplace=True)

# make dict in Major Charges
Feature_df['Major Charges']=list(make_dict_column(Feature_df))
Feature_df

,House,Measurement site,Sampling period,Measurement instruments,number of people per house,Number of rooms in the house,Major Charges,Start,End
0,House 1,Single family housing,5 minutes,Chauvin Arnoux 8335,3 adult people;,"3 quartos, 3 casas de banho, 1 sala de estar, ...","{'washer / dryer': '1', 'dishwasher': '1', 'el...",03-June-2011,18-June-2011
1,House 2,Single family housing,5 minutes,Chauvin Arnoux 8335,4 adult people;,"1 garage, 1 garden, 4 bedrooms, 2 bathrooms, 1...","{'washing machine': '1', 'dishwasher': '1', 'c...",16-July-2012,26-July-2012
2,House 3,Single family housing,5 minutes,Chauvin Arnoux 8335,2 adult people;,"3 bedrooms, 3 bathrooms, 1 living room, 1 kitc...","{'washer / dryer': '1', 'dishwasher': '1', 'el...",23-January-2013,07-February-2013
3,House 4,Single family housing,5 minutes,Chauvin Arnoux 8335,4 adult people;,"1 garage, 1 garden, 3 bedrooms, 2 bathrooms, 1...","{'pressure machine to wash': '1', 'washing mac...",06-January-2014,30-January-2014
4,House 5,Single family housing,5 minutes,Chauvin Arnoux 8335,3 adult people;,"2 bedrooms, 1 bathrooms, 1 living room/dining ...","{'washing machine': '1', 'electric oven': '1',...",04-August-2011,18-August-2011
5,House 7,Single student housing,5 minutes,Chauvin Arnoux 8335,2 adult people;,"2 rooms, 1 bathroom, 1 kitchen, 1 living room;","{'washer': '1', 'electric hob': '1', 'electric...",26-September-2011,03-October-2011
6,House 8,Single family housing,5 minutes,Chauvin Arnoux 8335,3 adult people;,"1 garage, 1 garden, 3 bedrooms, 2 bathrooms, 1...","{'pressure machine to wash': '1', 'washing mac...",01-June-2012,15-June-2012
7,House 9,Single family housing,5 minutes,Chauvin Arnoux 8335,3 adult people;,"1 garage, 1 garden, 3 bedrooms, 2 bathrooms, 1...","{'pressure machine to wash': '1', 'washing mac...",29-December-2012,12-January-2013
8,House 6,Single family housing,15 minutes,Chauvin Arnoux 8335,2 adult people and two people;,"2 quartos, 2 casas de banho, 1 sala de estar, ...",{'aN': 'N'},25-December-2011,15-March-2013
9,House 10,Single family housing,1 minutes,NaN,5 adult people;,"3 rooms, 3 bathrooms, 1 living room, 1 kitchen...",{'aN': 'N'},01-June-2012,30-June-2013


In [203]:

Feature_df.to_excel("Feature_houses.xls")

In [204]:
all_appliances_keys = []
[all_appliances_keys.append(list(
    Feature_df['Major Charges'][i].keys())) for i in range(len(Feature_df['Major Charges']))]

# make a long list of all the appliamces 
flatten = lambda l: [item for sublist in l for item in sublist]
all_appliances_keys_flat = flatten(all_appliances_keys)

# filter only the unique values of the appliances 
x = all_appliances_keys_flat

# stil 38 appliamces and overlap visable, so it needs to me cleaned at least til 10 
clean_keys(x)
        
# keep top 10 and labler other 5 in other     
unique_apliances = pd.DataFrame(x)
other_list_2 = list(unique_apliances[0].value_counts().index[10:])
other_list_2

renamed_appliences = make_other (x, other_list_2)

#count_plot = sns.barplot(x=unique_apliances[0].value_counts().index, y=unique_apliances[0].value_counts())
#count_plot.set_xticklabels(X, rotation=90)
unique_apliances[0].value_counts()
#renamed_appliences
#all_appliances_keys

tv                                    19
washing maschine                      14
laptop                                10
microwave                              8
fridge                                 8
cleaner                                7
heater                                 7
extractor                              6
kettle                                 5
dryer                                  4
electric oven                          4
electric hob                           4
freezer                                4
hair dryer                             3
electric fryer                         3
router                                 2
aN                                     2
toaster 1 extractor                    1
ompressor (three-phase)                1
nd tools (drill and angle grinder)     1
coffe machine                          1
Name: 0, dtype: int64

In [168]:
list(Feature_df['Major Charges'][0].keys())[0]

Feature_df['Major Charges'][0]['washing maschine'] = Feature_df['Major Charges'][0].pop(list(Feature_df['Major Charges'][0].keys())[0])

dict_i = Feature_df['Major Charges'][1]
key_j = list(dict_i.keys())[4]
tv = re.findall('TV|LCD', key_j)
tv
dict_i['tv'] = dict_i.pop(key_j)
dict_i

key_j



'tv'

In [180]:

unique_apliances_short_list = list(set(x))

#['microwave','cleaner','tv','OTHER','kettle','heater','fridge',
#'electric oven','washing maschine','extractor','laptop']


def from_dict_to_dummy (df):
    for i in range(len(df['Major Charges'])):
        # select dict 
        dict_i = df['Major Charges'][i]
        # select key 
        for j in range(len(list(dict_i.keys()))):
            key_j = list(dict_i.keys()[j]
            #pattern matching to see how to change the key 
            #tv = re.findall('TV|LCD', key_j)
            #laptop = re.findall('desktop computer|monitor|laptop', key_j)
            #cleaner = re.findall('cleaner', key_j )
            #kettle = re.findall('kettle', key_j )
            #wash = re.findall('wash', key_j)
            #heater = re.findall('heater', key_j)
            #fridge = re.findall('fridge', key_j) 
            # change key name
            #if tv != []: 
               # dict_i['tv'] = dict_i.pop(key_j)
            if laptop != []: 
                dict_i['laptop'] = dict_i.pop(key_j)
            elif cleaner != []: 
                dict_i['cleaner'] = dict_i.pop(key_j)
            elif kettle != []: 
                dict_i['kettle'] = dict_i.pop(key_j)
            elif wash != []: 
                dict_i['washing maschine'] = dict_i.pop(key_j)
            elif heater != []: 
                dict_i['heater'] = dict_i.pop(key_j)
            elif fridge != []: 
                dict_i['fridge'] = dict_i.pop(key_j)

    return df
            
from_dict_to_dummy(Feature_df)

SyntaxError: invalid syntax (<ipython-input-180-0af6138085cd>, line 25)